In [41]:
#import python extensions
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import sqlite3

In [42]:
#provide credetial, service file, and client
service_path = "/Users/peterkirgis/Downloads/"
service_file = 'wedge-project-peterkirgis-1e7b528def57.json' 
gbq_proj_id = 'wedge-project-peterkirgis'  
gbq_dataset_id = 'WedgeData2' 

private_key = service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [43]:
# run first query for sales by date by hour, read as DataFrame, write to textfile#
sales_date_hour = """SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(hour FROM datetime)) AS Hour,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-peterkirgis.WedgeData2.*` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND department != 20
    AND upc != '0'
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour"""
        
df = pd.read_gbq(sales_date_hour, project_id=gbq_proj_id)
df.to_csv('Summaryfiles/sales_date_hour', sep = ",", header = True, index = False)

Downloading: 100%|██████████| 38828/38828 [00:03<00:00, 9780.45rows/s]


In [44]:
# run second query for sales by owner by year by month, read as DataFrame, write to textfile#
sales_owner_year_month = """SELECT EXTRACT(year FROM datetime) as Year, 
        EXTRACT(month FROM datetime) as Month, 
        card_no as Owner,
        SUM(total) as total_sales, 
        COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
        SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
        FROM `wedge-project-peterkirgis.WedgeData2.*` 
        WHERE card_no != 3
        AND department != 0
        AND department != 15
        AND department != 20
        AND upc != '0'
        AND trans_status != 'M'
        AND trans_status != 'C'
        AND trans_status != 'J'
        AND (trans_status = ''
        OR trans_status = ' '
        OR trans_status =  'V'
        OR trans_status = 'R')
        GROUP BY Year, Month, card_no
        ORDER BY Year, Month, card_no"""
        
df = pd.read_gbq(sales_owner_year_month, project_id=gbq_proj_id)
df.to_csv('Summaryfiles/sales_owner_year_month', sep = ",", header = True, index = False)

Downloading: 100%|██████████| 808797/808797 [00:39<00:00, 20588.60rows/s]


In [45]:
# run third query for sales by description by year by month, read as DataFrame, write to textfile#
sales_description_year_month = """SELECT EXTRACT(year FROM datetime) as Year,
        EXTRACT(month FROM datetime) as Month, 
        upc as product_code, 
        description, 
        department, 
        CASE department 
        WHEN 1 THEN 'PACKAGED GROCERY' 
        WHEN 2 THEN 'PRODUCE' 
        WHEN 3 THEN 'BULK' 
        WHEN 4 THEN 'REF GROCERY' 
        WHEN 5 THEN 'CHEESE' 
        WHEN 6 THEN 'FROZEN' 
        WHEN 7 THEN 'BREAD' 
        WHEN 8 THEN 'DELI' 
        WHEN 9 THEN 'GEN MERCH' 
        WHEN 10 THEN 'SUPPLEMENTS' 
        WHEN 11 THEN 'PERSONAL CARE' 
        WHEN 12 THEN 'HERBS&SPICES' 
        WHEN 13 THEN 'MEAT' 
        WHEN 14 THEN 'JUICE BAR' 
        WHEN 16 THEN 'FISH&SEAFOOD' 
        WHEN 17 THEN 'BAKEHOUSE' 
        WHEN 18 THEN 'FLOWERS' 
        WHEN 19 THEN 'WEDGEWORLDWIDE' 
        WHEN 21 THEN 'CATERING' 
        WHEN 22 THEN 'BEER&WINE' END AS department_name,
        SUM(total) as total_sales, 
        COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
        SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
        FROM `wedge-project-peterkirgis.WedgeData2.*` 
        WHERE card_no != 3
        AND department != 0
        AND department != 15
        AND department != 20
        AND upc != '0'
        AND trans_status != 'M'
        AND trans_status != 'C'
        AND trans_status != 'J'
        AND (trans_status = ''
        OR trans_status = ' '
        OR trans_status =  'V'
        OR trans_status = 'R')
        GROUP BY Year, Month, upc, description, department
        ORDER BY Year, Month, upc, description, department"""
        
df = pd.read_gbq(sales_description_year_month, project_id=gbq_proj_id)
df.to_csv('Summaryfiles/sales_description_year_month', sep = ",", header = True, index = False)

Downloading: 100%|██████████| 1130383/1130383 [01:26<00:00, 13130.23rows/s]


In [46]:
#create sql database
db = sqlite3.connect("Wedge_Sum_Files.db")
cur = db.cursor()

In [47]:
#create schema for first query
cur.execute('''DROP TABLE IF EXISTS sales_date_hour''')
cur.execute('''CREATE TABLE sales_date_hour (
    Date DATE,
    Hour INTEGER,
    TotalSales INTEGER,
    Transactions INTEGER,
    Items INTEGER)''')

In [48]:
#import first query from textfile
with open('Summaryfiles/sales_date_hour','r', encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_date_hour (Date, Hour, TotalSales, Transactions, Items) VALUES (?,?,?,?,?)''', line)
db.commit()

In [49]:
#create schema for second query
cur.execute('''DROP TABLE IF EXISTS sales_owner_year_month''')
cur.execute('''CREATE TABLE sales_owner_year_month (
    Year INTEGER,
    Month INTEGER,
    Owner INTEGER,
    TotalSales INTEGER,
    Transactions INTEGER,
    Items INTEGER)''')

In [50]:
#import second query from textfile
with open('Summaryfiles/sales_owner_year_month','r', encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_owner_year_month (Year, Month, Owner, TotalSales, Transactions, Items) VALUES (?,?,?,?,?,?)''', line)
db.commit()

In [51]:
#create schema for third table
cur.execute('''DROP TABLE IF EXISTS sales_description_year_month''')
cur.execute('''CREATE TABLE sales_description_year_month (
    Year INTEGER,
    Month INTEGER,
    ProductCode VARCHAR,
    Description VARCHAR,
    Department INTEGER,
    DepartmentName VARCHAR,
    TotalSales INTEGER,
    Transactions INTEGER,
    Items INTEGER)''')

In [52]:
#import third table from textfile
with open('Summaryfiles/sales_description_year_month','r', encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_description_year_month (Year, Month, ProductCode, Description, Department, DepartmentName, TotalSales, Transactions, Items) VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()